In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
from scipy.special import gamma
from scipy.optimize import fsolve
from tqdm import tqdm
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.plotting import figure, show

next_infection_i = 1
lmb = 1.5
infected_indices = [0,1]
active_tau = [0,0]
L = []
b=2
a = 3
t_c = 0
for i in tqdm(range(10000)):
    Q = np.sort(np.random.exponential(3,4))
    f = lambda t: Q[0] - lmb*sum([1-np.exp(-((t-active_tau[k])/b)**a) for k in infected_indices])
    next_infection_tau = fsolve(f,t_c+b*((a-1)/a)**(1/a))
    if Q[0]<1.5:
        L.append(f(next_infection_tau[0]))
print(max(L)-min(L))    

In [2]:
def SIRSelke_Weibull_pulse(n,m,Q,lmb,a,b,T,P,p):
    """Function that runs a Sellke construction of a simple SIR model with a Weibull shaped infectivity profile. Includes a constant testing rate
    n - number of initial susceptibles
    m - number of initial infectives
    Q - n length vector with the individual thresholds of each of the susceptibles
    I - n+m length vector of the infectious periods of each of the individuals in the simulation"""
    A = 0 
    t = 0
    
    t_v = [0]
    A_v = [0]
    
    pulsed = False
    
    t_inf = [0]*(n+m)
    t_recov = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]

    
    active_tau = [0]*(n+m)
    
    for k in infected_indices:
        active_tau[k] = I[k]
    
    
    next_infection_i = m
    f = lambda x: Q[next_infection_i] - lmb*sum([1-np.exp(-((x-active_tau[k])/b)**a) for k in infected_indices])
    next_infection_tau = fsolve(f,t+b*((a-1)/a)**(1/a))

    while len(infected_indices)>0:
        #Decide whether next event is a recovery, infection or pulse
        A_check = A + (lmb/n)*next_recovery_tau*len(infected_indices)
        
        
        
        #print(next_recovery_tau)
        #input(active_tau)
        #print()
        
        if ((T-t)<min(next_recovery_tau,next_infection_tau)) and pulsed == False:
            #print("Pulse!")
            #print(infected_indices)
            #Pulse test
            dt = T-t
            A+= (lmb/n)*dt*len(infected_indices)
            t = T
            
            active_tau = [tau-dt for tau in active_tau]
            
            indices_spotted = []
            
           
            for i in infected_indices:
                if P[i]<p:
                    indices_spotted.append(i)
                    
                    active_tau[i] = np.inf
                    
                    
            for i in indices_spotted:
                recovered_indices.append(i)
                infected_indices.remove(i)
                t_recov[i] = t
                
                
            next_recovery_tau = min(active_tau)
            next_recovery_i = active_tau.index(next_recovery_tau)
            
            if len(infected_indices)>0:
                next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            
            pulsed = True
            #print(infected_indices)
        
        elif A_check<Q[next_infection_i-m]:
            #print("Recovery")
            #Next event is recovery  
            A += (lmb/n)*next_recovery_tau*len(infected_indices)
            
            t+= next_recovery_tau
            t_recov[next_recovery_i] = t
            active_tau = [tau-next_recovery_tau for tau in active_tau]
            active_tau = [np.inf if tau ==0 else tau for tau in active_tau]
                    
            
            recovered_indices.append(next_recovery_i)
            infected_indices.remove(next_recovery_i)
            
            next_recovery_tau = min(active_tau)
            next_recovery_i = active_tau.index(next_recovery_tau)
            
            if len(infected_indices)>0:
                next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            
        
            
        else:
            #print("Infection")
            #Next event is an infection
            next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            
            
                
            A+= (lmb/n)*next_infection_tau*len(infected_indices)
            
            t+=next_infection_tau
            
            t_inf[next_infection_i] = t
            active_tau = [tau-next_infection_tau for tau in active_tau]
            active_tau[next_infection_i] = I[next_infection_i]
            
            next_recovery_tau = min(active_tau)
            next_recovery_i = active_tau.index(next_recovery_tau)
            
            infected_indices.append(next_infection_i)
            susceptible_indices.remove(next_infection_i)
            
            next_infection_i += 1
            if len(infected_indices)>0:
                next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            
            if next_infection_i>=50:
                for i in infected_indices:
                    t_recov[i] = t+active_tau[i]
                    
                break
                
                
                
                
                
                
                
        t_v.append(t)
        A_v.append(A)
        #print("t=",t)
        #print("A =",A) 
        #print(active_tau)
        #print("")


    return(t_v,A_v,t_inf,t_recov,recovered_indices)
            

In [3]:
def SIRSelke(n,m,Q,I,lmb):
    """Function that runs a Sellke construction of a simple SIR model with a constant infectivity
    n - number of initial susceptibles
    m - number of initial infectives
    Q - n length vector with the individual thresholds of each of the susceptibles
    I - n+m length vector of the infectious periods of each of the individuals in the simulation"""
    A = 0 
    t = 0
    
    
    t_inf = [0]*(n+m)
    t_recov = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]
    recovered_indices = []
    
    active_tau = [np.inf]*(n+m)
    
    for k in infected_indices:
        active_tau[k] = I[k]
    

    
    next_recovery_tau = min(active_tau)
    next_recovery_i = active_tau.index(next_recovery_tau)
    
    next_infection_i = m

    while len(infected_indices)>0:
        #Decide whether next event is a recovery or an infection
        A_check = A + (lmb/n)*next_recovery_tau*len(infected_indices)
        
        
        
        if A_check<Q[next_infection_i-m]:
            #print("Recovery")
            #Next event is recovery  
            A += (lmb/n)*next_recovery_tau*len(infected_indices)
            
            t+= next_recovery_tau
            t_recov[next_recovery_i] = t
            active_tau = [tau-next_recovery_tau for tau in active_tau]
            active_tau = [np.inf if tau ==0 else tau for tau in active_tau]
                    
            
            recovered_indices.append(next_recovery_i)
            infected_indices.remove(next_recovery_i)
            
            next_recovery_tau = min(active_tau)
            next_recovery_i = active_tau.index(next_recovery_tau)
            
        else:
            #Next event is an infection
            tau_of_inf = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            A+= (lmb/n)*tau_of_inf*len(infected_indices)
            
            t+=tau_of_inf
            
            t_inf[next_infection_i] = t
            active_tau = [tau-tau_of_inf for tau in active_tau]
            active_tau[next_infection_i] = I[next_infection_i]
            
            next_recovery_tau = min(active_tau)
            next_recovery_i = active_tau.index(next_recovery_tau)
            
            infected_indices.append(next_infection_i)
            susceptible_indices.remove(next_infection_i)
            
            next_infection_i += 1
            
            if next_infection_i>=50:
                for i in infected_indices:
                    t_recov[i] = t+active_tau[i]
                    
                break
            

    return(recovered_indices)
            

In [4]:
def SIRSelke_constant_pulse(n,m,Q,I,lmb,T,P,p):
    """Function that runs a Sellke construction of a simple SIR model with a constant infectivity. Includes a constant testing rate
    n - number of initial susceptibles
    m - number of initial infectives
    Q - n length vector with the individual thresholds of each of the susceptibles
    I - n+m length vector of the infectious periods of each of the individuals in the simulation"""
    A = 0 
    t = 0
    
    t_v = [0]
    A_v = [0]
    
    pulsed = False
    
    t_inf = [0]*(n+m)
    t_recov = [0]*(n+m)

    susceptible_indices = [k for k in range(m,n+m)]
    infected_indices = [k for k in range(m)]
    recovered_indices = []
    
    active_tau = [np.inf]*(n+m)
    
    for k in infected_indices:
        active_tau[k] = I[k]
    

    
    next_recovery_tau = min(active_tau)
    next_recovery_i = active_tau.index(next_recovery_tau)
    
    next_infection_i = m
    next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))

    while len(infected_indices)>0:
        #Decide whether next event is a recovery, infection or pulse
        A_check = A + (lmb/n)*next_recovery_tau*len(infected_indices)
        
        
        
        #print(next_recovery_tau)
        #input(active_tau)
        #print()
        
        if ((T-t)<min(next_recovery_tau,next_infection_tau)) and pulsed == False:
            #print("Pulse!")
            #print(infected_indices)
            #Pulse test
            dt = T-t
            A+= (lmb/n)*dt*len(infected_indices)
            t = T
            
            active_tau = [tau-dt for tau in active_tau]
            
            indices_spotted = []
            
           
            for i in infected_indices:
                if P[i]<p:
                    indices_spotted.append(i)
                    
                    active_tau[i] = np.inf
                    
                    
            for i in indices_spotted:
                recovered_indices.append(i)
                infected_indices.remove(i)
                t_recov[i] = t
                
                
            next_recovery_tau = min(active_tau)
            next_recovery_i = active_tau.index(next_recovery_tau)
            
            if len(infected_indices)>0:
                next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            
            pulsed = True
            #print(infected_indices)
        
        elif A_check<Q[next_infection_i-m]:
            #print("Recovery")
            #Next event is recovery  
            A += (lmb/n)*next_recovery_tau*len(infected_indices)
            
            t+= next_recovery_tau
            t_recov[next_recovery_i] = t
            active_tau = [tau-next_recovery_tau for tau in active_tau]
            active_tau = [np.inf if tau ==0 else tau for tau in active_tau]
                    
            
            recovered_indices.append(next_recovery_i)
            infected_indices.remove(next_recovery_i)
            
            next_recovery_tau = min(active_tau)
            next_recovery_i = active_tau.index(next_recovery_tau)
            
            if len(infected_indices)>0:
                next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            
        
            
        else:
            #print("Infection")
            #Next event is an infection
            next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            
            
                
            A+= (lmb/n)*next_infection_tau*len(infected_indices)
            
            t+=next_infection_tau
            
            t_inf[next_infection_i] = t
            active_tau = [tau-next_infection_tau for tau in active_tau]
            active_tau[next_infection_i] = I[next_infection_i]
            
            next_recovery_tau = min(active_tau)
            next_recovery_i = active_tau.index(next_recovery_tau)
            
            infected_indices.append(next_infection_i)
            susceptible_indices.remove(next_infection_i)
            
            next_infection_i += 1
            if len(infected_indices)>0:
                next_infection_tau = (Q[next_infection_i-m]-A)/((lmb/n)*len(infected_indices))
            
            if next_infection_i>=50:
                for i in infected_indices:
                    t_recov[i] = t+active_tau[i]
                    
                break
                
                
                
                
                
                
                
        t_v.append(t)
        A_v.append(A)
        #print("t=",t)
        #print("A =",A) 
        #print(active_tau)
        #print("")


    return(t_v,A_v,t_inf,t_recov,recovered_indices)
            